In [1]:
import numpy as np
from DMD import DMD
import matplotlib as mpl
import matplotlib.pyplot as plt
import os,time,sys
import h5py
import scipy as sci
import scipy.spatial.distance as ssd
import parseCIFTI as pC
from glob import glob
import h5py
import copy
from tqdm import tqdm_notebook as tqdm
from sklearn.decomposition import FastICA

#import utility functions. Notably includes:
#'binCortexData(x,nb=40)': take array of grayordinate data and convert to (flattened) binned space (split into nb bins)
#'flat2mat(x)': take flattened, binned vector and convert to matrix for visualization
#'mat2flat(x)': take matrix and convert to flat
#'initR()': "R" is a necessary global variable which gets defined in the above functions, this is a workaround to make sure it's defined; this should be changed
from hcp_dmd_utils import *
initR()

%matplotlib inline

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Load gICA Modes

In [2]:
import nibabel as nib
import copy
from tqdm import tqdm
dL,dR,idx,scb,scl=pC.parseCoords()
nSC=dL.shape[0]+dR.shape[0]
u120=np.genfromtxt('./u120.txt').astype(int).astype(str)
ulist=u120

fdir='/hdd/HCP_DMD/HCP_DMD/ICA/3T_HCP1200_MSMAll_d15_ts2_Z/'
DMN_ICA=[]
for u in tqdm(ulist):
    data=nib.load(fdir+'{:}.dtseries.nii'.format(u))
    data=np.array(data.get_data())[0,0,0,0,:,:]
    IC=binCortexData(data.T)[:,1]
    IC=(IC-IC.mean())/IC.std()
    DMN_ICA.append(IC)
DMN_ICA=np.vstack(DMN_ICA)

100%|██████████| 120/120 [00:04<00:00, 24.33it/s]


# Run sDMD

In [ ]:
#%% EXPERIMENT TO GET BEST PARAMETERS FOR EXTRACTING DMN
scandirs=['LR','RL']
runlabels=['REST1','REST2']
nmodes=8
nframes=32


mthresh=25
zthresh=2.5
min_clust_size=5

zthreshes=np.linspace(1.5,3,61)
fct_list=np.linspace(0.65,0.98,101)

#zthreshes=np.linspace(2.14,2.18,3)
#fct_list=np.linspace(0.75,0.85,41)

try:
    os.mkdir('./sDMDvGICA')
except:
    pass
try:
    os.mkdir('./sDMDvGICA/params')
except:
    pass

with h5py.File('./RSN.h5','r') as hf:
  RSNs=np.array(hf['RSNs'])
  RSNlabels=np.array(hf['RSNlabels'])
rsn=[]        
for k in range(RSNs.shape[2]):
  r=RSNs[:,:,k]
  rsn.append(mat2flat(r))
rsn=np.vstack(rsn)
rsnNo=[3,4,9,13]
rsn=rsn[rsnNo,:]
rlabs=RSNlabels[rsnNo] 

#for uk,u in enumerate([ulist[0]]):
for uk,u in enumerate(ulist):
  print(u)
  try:
    icak=DMN_ICA[uk]
    dmn_best=[]
    Cor=np.zeros((len(zthreshes),len(fct_list)))
    F=[]
    T=[]
    try:
      for runlabel in runlabels:
        for scandir in scandirs:
          fdir='./DMD_results/nmodes{:}_nframes{:}_{:}_{:}/'.format(nmodes,nframes,runlabel,scandir)
          with h5py.File(fdir+'Modes_{:}.h5'.format(u),'r') as hf:
            F.append(np.array(hf['F']))
    except:
      pass
      
    
    F=np.vstack(F)
    
    Mflat=(F-np.mean(F,1)[:,None])/F.std(1)[:,None]
     
    mpro=[]
    for f in Mflat:
      m=flat2mat(f>2.0).astype(float)
      mpro.append(np.sum(np.sum(m[1:,1:]*m[:-1,:-1],0),0))
    mpro=np.hstack(mpro) 
     
    mwhere=mpro>mthresh
    
    import scipy.cluster.hierarchy as sch
    Fz=F[mwhere,:]
    Fz=(Fz-np.mean(Fz,1)[:,None])/np.std(Fz,1)[:,None]
    
    for i,zthresh in enumerate(tqdm(zthreshes)):
      Fn=(Fz>zthresh).astype(float)
      method='average';metric='correlation';
      
      Z=sch.linkage(Fn, method, metric)
      ix=sch.dendrogram(Z,no_plot=True)['leaves']  
      for j,fclust_thresh in enumerate(fct_list):
        try:
          cgroups=sch.fcluster(Z,fclust_thresh,'distance')[ix]
          cgroups=cgroups.astype(float)
          cun,count=np.unique(cgroups,return_counts=True)
          for k,c in enumerate(cun):
              if count[k]<min_clust_size:
                  cgroups[cgroups==c]=-1
                  
          cgroups=np.unique(cgroups,return_inverse=True)[1].astype(float)
          cgroups[cgroups==0]=np.nan
          cgroups+=-1
          
          cif=np.isfinite(cgroups)
          cuni=np.unique(cgroups[cif])
          
          cbig=np.ones(cgroups.shape)*np.nan
          ck=-1
          for c in cuni:
              if np.sum(cgroups==c)>1:
                  ck+=1
                  cbig[cgroups==c]=ck
          
          glist=np.unique(cgroups[np.isnan(cgroups)==False]).astype(int)
          
          modes=F[mwhere][ix][cif]
          
          moda=[]
          for c in cuni:
            moc=modes[cbig[cif]==c].mean(0)
            moc=(moc-moc.mean())/moc.std()
            moda.append(moc)
            
          moda=np.vstack(moda)
          #phasa=np.vstack(phasa)
          
          dmn=moda[np.argmax(np.corrcoef(rsn[0],moda)[0][1:])]
          
          #cscore=np.corrcoef(dmn,DMN_ICA)[0,1:][uk]
          cscore=np.corrcoef(dmn,icak)[0][1]
          #print '{:0.2f},{:0.2f}: correlation={:0.3f} / {:0.3f}'.format(zthresh,fclust_thresh,cscore[ui],np.mean(cscore))
          
          Cor[i,j]=cscore
          
          if cscore==Cor.max():
            dmn_best=copy.copy(dmn)
            fc_best=copy.copy(fclust_thresh)
            zthresh_best=copy.copy(zthresh)
        except:
          pass

    Db=(dmn_best-dmn_best.mean())/dmn_best.std()
    Db=flat2mat(Db)
    Db[Db==0]=np.nan  
    Ik=(icak-icak.mean())/icak.std()
    Ik=flat2mat(Ik)
    Ik[Ik==0]=np.nan

    Cpop=list(np.corrcoef(dmn_best,DMN_ICA)[0,1:])
    Csub=Cpop.pop(uk)
    Cpop=np.array(Cpop)
    
    with h5py.File('./sDMDvGICA/params/{:}.h5'.format(u),'w') as hf:
      hf.create_dataset('Cor',data=Cor)
      hf.create_dataset('zthreshes',data=zthreshes)
      hf.create_dataset('fct_list',data=fct_list)
      hf.create_dataset('mthresh',data=mthresh)
      hf.create_dataset('Db',data=Db)
      hf.create_dataset('Ik',data=Ik)
      hf.create_dataset('dmn',data=dmn_best)
      hf.create_dataset('Cpop',data=Cpop)
      hf.create_dataset('Csub',data=Csub)
      hf.create_dataset('uk',data=uk)
      hf.create_dataset('fc_best',data=fc_best)
      hf.create_dataset('zthresh_best',data=zthresh_best)
    tqdm.write('\n Best correlations: {:0.3f}, {:0.3f}'.format(Csub,Cpop.mean()))

  except (KeyboardInterrupt, SystemExit):
    raise
  except:
    raise
    

In [ ]:

from sklearn.decomposition import FastICA
from tqdm import tqdm

flag='_best_spatial'

dL,dR,idx,scb,scl=pC.parseCoords()
nSC=dL.shape[0]+dR.shape[0]
u120=np.genfromtxt('./u120.txt').astype(int).astype(str)
ulist=u120
scandirs=['LR','RL']
runlabels=['REST1','REST2']

#nmodes=[8,12,16,24,32]
nmodes=[8,10,12,16]
npoint=dR.shape[0]+dL.shape[0]

for uk,u in enumerate(ulist):
 if os.path.isfile('./sDMDvGICA/params/ica_{:}{:}.png'.format(u,flag)):
  print '{:} already calculated!'.format(u)
 else:
  tic=time.time()
  X=[]
  for runlabel in runlabels:
    for scandir in scandirs:
      try:
        fname='/hdd/HCP_DMD/HCP_DMD/data/'+u+'/rfMRI_'+runlabel+'_'+scandir+'_Atlas.dtseries.nii'
        X.append(pC.loadGrayord(fname)[:npoint])
      except:
        pass
  X=np.hstack(X)
  bestcscore=0.0
  #nica=5
  nica=3
  for nk in tqdm(range(nica)):
    try:

      #ica=FastICA(np.random.choice(nmodes),max_iter=1000).fit(X.T).components_.T
      ica=FastICA(np.random.choice(nmodes),max_iter=1000).fit_transform(X)
      
      
      I=binCortexData(ica)
      I=(I-I.mean(0)[None,:])/I.std(0)[None,:]
      cscore_list=np.corrcoef((DMN_ICA[uk]>2).astype(float),(I.T>2).astype(float))[0,1:]
      cscore=np.max(cscore_list)
      
      if cscore>bestcscore:
        bestcscore=cscore
        icadmn=I[:,np.argmax(cscore_list)]
        icadmn=(icadmn-icadmn.mean())/icadmn.std()       
        Cpop=list(np.corrcoef((icadmn>2.0).astype(float),(DMN_ICA>2.0).astype(float))[0,1:])
        Csub=Cpop.pop(uk)
        Cpop=np.array(Cpop)
        
    
    except:
      pass
    
  with h5py.File('./sDMDvGICA/params/{:}.h5'.format(u),'r') as hf:
    #dmd_Cpop=np.array(hf['Cpop'])
    #dmd_Csub=np.array(hf['Csub'])
    D=np.array(hf['Db'])
    GI=np.array(hf['Ik'])
    
  dmddmn=mat2flat(D)
  
  dmd_cpop=list(np.corrcoef((dmddmn>2.0).astype(float),(DMN_ICA>2.0).astype(float))[0,1:])
  dmd_csub=dmd_cpop.pop(uk)
  dmd_cpop=np.array(dmd_cpop)
  
  with h5py.File('./sDMDvGICA/params/{:}_ica{:}.h5'.format(u,flag),'w') as hf:
    hf.create_dataset('Csub_ica',data=Csub)
    hf.create_dataset('Cpop_ica',data=Cpop)
    hf.create_dataset('Csub_dmd',data=dmd_csub)
    hf.create_dataset('Cpop_dmd',data=dmd_cpop)
    hf.create_dataset('icadmn',data=icadmn)
    hf.create_dataset('nmodes',data=nmodes)

  
  
  I=flat2mat(icadmn)
  I[I==0.0]=np.nan
  #%
  plt.close('all')
  plt.figure(figsize=(14,5))
  #plt.get_current_fig_manager().window.setGeometry(295, 260, 1418, 554)
  plt.subplot(1,3,1)
  plt.imshow(I,vmin=0,vmax=2,cmap='viridis')
  plt.title('ICA: {:0.3f}/{:0.3f}'.format(Csub,Cpop.mean()))
  plt.xticks([]);plt.yticks([]);
  plt.subplot(1,3,2)
  plt.imshow(D,vmin=0,vmax=2,cmap='viridis')
  plt.title('DMD: {:0.3f}/{:0.3f}'.format(float(dmd_csub),dmd_cpop.mean()))
  plt.xticks([]);plt.yticks([]);
  plt.subplot(1,3,3)
  plt.imshow(GI,vmin=0,vmax=2,cmap='viridis')
  plt.xticks([]);plt.yticks([])
  
  plt.savefig('./sDMDvGICA/params/ica_{:}{:}.png'.format(u,flag))
  print '{:}, {:} modes: {:0.3f}/{:0.3f}'.format(u,nmodes,Csub,Cpop.mean())
  print 'Time elapsed is {:0.1f} seconds'.format(time.time()-tic)
  
  

pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
 67%|██████▋   | 2/3 [05:24<02:42, 162.23s/it]/usr/local/lib/python2.7/dist-packages/sklearn/decomposition/fastica_.py:116: UserWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn('FastICA did not converge. Consider increasing '
100%|██████████| 3/3 [08:19<00:00, 165.98s/it]
pixdim[1,2,3] should be non-zero; setting 0 dims to 1


102513, [8, 10, 12, 16] modes: 0.590/0.462
Time elapsed is 501.9 seconds


pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
100%|██████████| 3/3 [08:05<00:00, 161.98s/it]
pixdim[1,2,3] should be non-zero; setting 0 dims to 1


104012, [8, 10, 12, 16] modes: 0.484/0.374
Time elapsed is 501.0 seconds


pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
100%|██████████| 3/3 [08:03<00:00, 161.14s/it]
pixdim[1,2,3] should be non-zero; setting 0 dims to 1


...

pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
100%|██████████| 3/3 [07:54<00:00, 158.13s/it]
pixdim[1,2,3] should be non-zero; setting 0 dims to 1


223929, [8, 10, 12, 16] modes: 0.038/0.012
Time elapsed is 490.5 seconds


pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
100%|██████████| 3/3 [08:22<00:00, 166.78s/it]
pixdim[1,2,3] should be non-zero; setting 0 dims to 1


227432, [8, 10, 12, 16] modes: 0.398/0.376
Time elapsed is 518.4 seconds


pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
import numpy as np
import h5py
import pandas as pd
u120=np.genfromtxt('/hdd/HCP_DMD/HCP_DMD/u120.txt').astype(int).astype(str)
ulist=u120
ica_pop=[];ica_sub=[]
dmd_pop=[];dmd_sub=[]
for u in ulist:
  try:
    with h5py.File('./sDMDvGICA/params/{:}_ica_best_spatial.h5'.format(u),'r') as hf:
      ica_pop.append(np.array(hf['Cpop_ica']))
      ica_sub.append(np.array(hf['Csub_ica']))
      dmd_pop.append(np.array(hf['Cpop_dmd']))
      dmd_sub.append(np.array(hf['Csub_dmd']))
  except:
    pass
dmd_pop=np.hstack(dmd_pop)
dmd_sub=np.hstack(dmd_sub)
ica_pop=np.hstack(ica_pop)
ica_sub=np.hstack(ica_sub)

dlist=[dmd_pop,dmd_sub,ica_pop,ica_sub]
dlabs=np.hstack([k*np.ones(len(d)) for k,d in enumerate(dlist)]).astype(int)
data=np.hstack(dlist)
is_ica=np.array(dlabs>=2).astype(int)
icatabs=np.array(['sDMD','ICA'])
is_ica=icatabs[is_ica]

dlt=['Across Population','Within Same Subject']
dlabtabs=np.array(dlt*2)
dlabs=dlabtabs[dlabs]

corlab='Correlation of DMD/ICA DMNs with gICA DMNs'

df=pd.DataFrame.from_dict({'popcompare':dlabs,corlab:data,'isICA':is_ica})

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
plt.close('all')
plt.figure()
plt.get_current_fig_manager().window.setGeometry(594, 261, 1155, 394)
#g=sns.violinplot(data=df,x='popcompare',y='Correlation with gICA DMN',hue='isICA',inner='quartile',split=True,legend_out=True)
plt.plot([0,0.7],[0,0],c=[0.3,0.3,0.3])
plt.plot([0,0.7],[1,1],c=[0.3,0.3,0.3])
g=sns.violinplot(data=df,y='popcompare',x=corlab,hue='isICA',inner='quartile',split=True,legend_out=True,scale='width')
leg=g.axes.get_legend()
leg.set_title('');leg._loc=2

plt.xlim([0,0.7])
#plt.xlabel('')
plt.yticks([0,1],['across   \n Population','within \n Subject'],rotation=90,va='center',size=16)
plt.ylabel('')
plt.xlabel('Correlation',size=16)

###SAVE
#plt.savefig('./sDMDvGICA/Distributions.svg')
